In [ ]:
#Import libraries

import rasterio
import geopandas as gpd
import numpy as np
from rasterio.mask import mask
from shapely.geometry import Polygon, MultiPolygon


In [ ]:
# DATASET
Built-up='add your built-up geotiff layer'
building_polygon='add your polygon shapefile'

In [ ]:
#RBF Application
# Load the land cover raster
with rasterio.open(Built-up) as src:
    raster = src.read(1)  # Assuming the raster data is in the first band
    affine = src.transform
    crs = src.crs  # Coordinate reference system

# Load the building polygon shapefile
shapefile_path = building_polygon
buildings = gpd.read_file(shapefile_path)

# Reproject buildings to the same CRS as the raster, if necessary
if buildings.crs != crs:
    buildings = buildings.to_crs(crs)

# Filter out invalid geometries (if any)
buildings = buildings[buildings.geometry.notnull()]

# Ensure only valid Polygon or MultiPolygon geometries are used
valid_geometries = buildings[buildings.geometry.apply(lambda g: isinstance(g, (Polygon, MultiPolygon)))]

# Convert the valid geometries to GeoJSON format for rasterio masking
geometries = [feature['geometry'] for feature in valid_geometries.__geo_interface__['features']]

# Apply the mask to extract only the pixels within the valid building polygons
with rasterio.open(Built-up) as src:
    out_image, out_transform = mask(src, geometries, crop=True)
    out_meta = src.meta

# Extract only the built-up pixels: Depend on built-up layer property. 
# Some could use different pixel value
built_up_pixels = np.where(out_image == 1, out_image, 0)


# Save the masked raster with built-up area pixels
out_meta.update({"driver": "GTiff", "height": built_up_pixels.shape[1], "width": built_up_pixels.shape[2], "transform": out_transform})

RBF_raster = 'At your saving path*.tif'
with rasterio.open(RBF_raster, 'w', **out_meta) as dest:
    dest.write(built_up_pixels)

print("Built-up area pixels have been extracted and saved.")